In [ ]:
!pip install yfinance
!pip install ta

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import warnings
import ta
warnings.filterwarnings("ignore")

In [ ]:
# The code here will allow you to switch your graphics to dark mode for those who choose to code in dark mode
import matplotlib.pyplot as plt

import matplotlib as mpl
from matplotlib import cycler
colors = cycler('color',
                ['#669FEE', '#66EE91', '#9988DD',
                 '#EECC55', '#88BB44', '#FFBBBB'])
plt.rc('figure', facecolor='#313233')
plt.rc('axes', facecolor="#313233", edgecolor='none',
       axisbelow=True, grid=True, prop_cycle=colors,
       labelcolor='gray')
plt.rc('grid', color='474A4A', linestyle='solid')
plt.rc('xtick', color='gray')
plt.rc('ytick', direction='out', color='gray')
plt.rc('legend', facecolor="#313233", edgecolor="#313233")
plt.rc("text", color="#C9C9C9")

# Manage the data

In [ ]:
# Preprocessing function

def preprocessing(name):
  
  # Import the data
  df = pd.read_csv(name, delimiter="\t", index_col="<DATE>", parse_dates=True)

  # Delete the two last columns
  df = df.iloc[:,:-2]

  # Rename
  df.columns = ["open", "high", "low", "close", "volume"]
  df.index.name = "time"

  return df

In [ ]:
def preprocessing_yf(symbol):
  
  #Import the data
  df = yf.download("EURUSD=X").dropna()

  #Rename
  df.columns = ["open", "high", "low", "close", "adj close", "volume"]
  df.index.name = "time"

  # Remove adj close
  del df["adj close"]

  return df
df = preprocessing_yf("EURUSD=X")
df

# Moving average /RSI

In [ ]:
# Create Resistance using a rolling max
df["SMA fast"] = df["close"].rolling(30).mean()

# Create Support using a rolling min
df["SMA slow"] = df["close"].rolling(60).mean()


# Create RSI
df["rsi"] = ta.momentum.RSIIndicator(df["close"], window=10).rsi()

In [ ]:
# Plot the results
df[["close", "SMA fast", "SMA slow"]].loc["2020"].plot(figsize=(15,8))

In [ ]:
df["rsi"].loc["2020"].plot(figsize=(15,8))

# Strategy

In [ ]:
df["position"] = 0

# RSI yersteday
df["rsi yersteday"] = df["rsi"].shift(1)

# Conditions
condition_1_buy = df["SMA fast"] > df["SMA slow"]
condition_2_buy = df["rsi"] < df["rsi yersteday"]

condition_1_sell = df["SMA fast"] < df["SMA slow"]
condition_2_sell = df["rsi"] > df["rsi yersteday"]

# Create the condition
df.loc[condition_1_buy & condition_2_buy, "position"] = 1
df.loc[condition_1_sell & condition_2_sell, "position"] = -1

# Verification Graph

In [ ]:
# We plot all the signla to be sure that they be correct

year="2019-01"

# Select all signal in a index list to plot only this points
idx_open = df.loc[df["position"] == 1].loc[year].index
idx_close = df.loc[df["position"] == -1].loc[year].index



# Adapt the size of the graph
plt.figure(figsize=(15,6))

# Plot the points of the open long signal in green and sell in red
plt.scatter(idx_open, df.loc[idx_open]["close"].loc[year], color= "#57CE95", marker="^")
plt.scatter(idx_close, df.loc[idx_close]["close"].loc[year], color= "red", marker="v")


# Plot the resistance to be sure that the conditions are completed
plt.plot(df["close"].loc[year].index, df["close"].loc[year], alpha=0.35)

plt.plot(df["close"].loc[year].index, df["SMA fast"].loc[year], alpha=0.35)

plt.plot(df["close"].loc[year].index, df["SMA slow"].loc[year], alpha=0.35)



# Show the graph
plt.show()
plt.close()


plt.figure(figsize=(15,8))
plt.scatter(idx_open, df.loc[idx_open]["rsi"].loc[year], color= "#57CE95", marker="^")
plt.scatter(idx_close, df.loc[idx_close]["rsi"].loc[year], color= "red", marker="v")

plt.plot(df["rsi"].loc[year].index, df["rsi"].loc[year], alpha=0.35)

plt.show()

# Compute the profit

In [ ]:
# Compute the percentage of variation of the asset
df["pct"] = df["close"].pct_change(1)

# Compute the return of the strategy
df["return"] = df["pct"] * (df["position"].shift(1))

df["return"].cumsum().plot(figsize=(15,8))


# Add Stop loss


In [ ]:
# % = (SV - EV) / EV
df["Min"] = (df["low"] - df["close"].shift(1)) / df["close"].shift(1)

In [ ]:
# -------- Prepare the data ----------
# Remove missing values and order them
dfh = df[["Min"]].loc["2005":"2015"].dropna().sort_values(by="Min", ascending=True)

dfh.loc[df["Min"]<-0.3] = 0

# Weight for each value
dfh["probability"] = 1/len(dfh)


# -------- Plot the graph ----------
# Adapt the size
plt.figure(figsize=(20,8))

# Put a ylabel
plt.ylabel("P(X<x)", size=15)

# Put a xlabel
plt.xlabel("Strategy's daily low in %", size=15)

# Put a title
plt.title("CDF of the daily low", size=20)

# Plot the line of the CDF
plt.plot(dfh["Min"]*100, dfh["probability"].cumsum(), color="#AB371E")

# Plot the area of the CDF
plt.fill_between(dfh["Min"]*100,0, dfh["probability"].cumsum(), color="#AB371E", alpha=0.35)

# Define x limits
plt.xlim([-3.5, 0])

# Plot the graph
plt.show()

In [ ]:
dfh

In [ ]:
df["return SL"] = df["return"]
sl = 0.10
df.loc[df["Min"]<-sl, "return SL"] = -sl

In [ ]:
# Train 
df["return"].loc[:"2015"].cumsum().plot(figsize=(15,8))
df["return SL"].loc[:"2015"].cumsum().plot(figsize=(15,8), color="red")

In [ ]:
# Test 
df["return"].loc["2015":].cumsum().plot(figsize=(15,8))
df["return SL"].loc["2015":].cumsum().plot(figsize=(15,8), color="red")


# Automatization

In [ ]:
def preprocessing_yf(symbol):
  df = yf.download(symbol).dropna()
  df.columns = ["open", "high", "low", "close", "adj close", "volume"]
  return df

def preprocessing(name):

  # Import the data
  df = pd.read_csv(f"{name}", delimiter="\t", index_col="<DATE>", parse_dates=True)

  # Delete the two last columns
  df = df.iloc[:,:-2]

  # Rename
  df.columns = ["open", "high", "low", "close", "volume"]
  df.index.name = "name"

  return df
  
def SMA_strategy(input, mt5=False, yf=False):

  if mt5:
    df = preprocessing(input)
  
  if yf:
    df = preprocessing_yf(input)

  
  # Create Resistance using a rolling max
  df["SMA fast"] = df["close"].rolling(30).mean()

  # Create Support using a rolling min
  df["SMA slow"] = df["close"].rolling(60).mean()

  # Create RSI
  df["rsi"] = ta.momentum.RSIIndicator(df["close"], window=10).rsi()


  df["position"] = 0

  df["rsi yersteday"] = df["rsi"].shift(1)
  # Create the condition
  df.loc[(df["SMA fast"] > df["SMA slow"]) & (df["rsi"] < df["rsi yersteday"]), "position"] = 1
  df.loc[(df["SMA fast"] < df["SMA slow"]) & (df["rsi"] > df["rsi yersteday"]), "position"] = -1

  

  df["pct"] = df["close"].pct_change(1)

  # Compute the return of the strategy

  df["return"] = df["pct"] * (df["position"].shift(1))
  

  return df["return"]

In [ ]:
SMA_strategy("EURUSD=X", yf=True).cumsum().plot(figsize=(15,8))

In [ ]:
SMA_strategy("EURUSD_D1.csv", mt5=True).cumsum().plot(figsize=(15,8))

In [ ]:
# Same strategy but with different assets
yahoo = SMA_strategy("EURUSD=X", yf=True)
metatrader = SMA_strategy("EURUSD_D1.csv", mt5=True)

returns = pd.DataFrame([yahoo, metatrader], index=["Yahoo",
                                                   "Broker"]).transpose().dropna().cumsum(axis=0)

# Plot on graph

# Adapt the size
plt.figure(figsize=(20,8))

# Plot the returns
plt.plot(returns["Yahoo"]*100, label="Yahoo")
plt.plot(returns["Broker"]*100, label="MetaTrader")

# Plot title + name axis 
plt.xlabel("Time", size=15)
plt.ylabel("Profits % ", size=15)
plt.title("Difference between strategies on the same asset but using different data", size=20)


# Plot legend 
plt.legend()
plt.show()

# Portfolio

In [ ]:
namelist = ["GOOG", "MSFT", "BTC-USD", "ETH-USD", "EURUSD=X", "GBPCAD=X"]
returns = []
for name in namelist:
  ret = SMA_strategy(name, yf=True)
  returns.append(ret)
  ret.cumsum().plot(figsize=(15,8))
  plt.show()
  plt.close()

In [ ]:
pf = pd.DataFrame(returns, index=namelist).transpose().dropna()

In [ ]:
ret_pf = (pf.sum(axis=1)/len(namelist))
ret_pf.name="return"
ret_pf.cumsum(axis=0).plot()

In [ ]:
def drawdown_function(serie):

  # We compute Cumsum of the returns
  cum = serie.dropna().cumsum() + 1

  # We compute max of the cumsum on the period (accumulate max) # (1,3,5,3,1) --> (1,3,5,5,5)
  running_max = np.maximum.accumulate(cum)

  # We compute drawdown
  drawdown = cum/running_max - 1
  return drawdown
def BackTest(serie, annualiazed_scalar=252):

  # Import the benchmark
  sp500 = yf.download("^GSPC")["Adj Close"].pct_change(1)
  
  # Change the name
  sp500.name = "SP500"

  # Concat the returns and the sp500
  val = pd.concat((serie,sp500), axis=1).dropna()
  # Compute the drawdown
  drawdown = drawdown_function(serie)*100
  
  # Compute max drawdown
  max_drawdown = -np.min(drawdown)




  # Put a subplots
  fig, (cum, dra) = plt.subplots(1,2, figsize=(20,6))
  
  # Put a Suptitle
  fig.suptitle("Backtesting", size=20)

  # Returns cumsum chart
  cum.plot(serie.cumsum()*100, color="#39B3C7")

  # SP500 cumsum chart
  cum.plot(val["SP500"].cumsum()*100, color="#B85A0F")

  # Put a legend
  cum.legend(["Portfolio", "SP500"])
  
  # Set individual title
  cum.set_title("Cumulative Return", size=13)

  cum.set_ylabel("Cumulative Return %", size=11)

  # Put the drawdown
  dra.fill_between(drawdown.index,0,drawdown, color="#C73954", alpha=0.65)

  # Set individual title
  dra.set_title("Drawdown", size=13)

  dra.set_ylabel("drawdown in %", size=11)

  # Plot the graph
  plt.show()


  # Compute the sortino
  sortino = np.sqrt(annualiazed_scalar) * serie.mean()/serie.loc[serie<0].std()

  # Compute the beta
  beta = np.cov(val[["return", "SP500"]].values,rowvar=False)[0][1] / np.var(val["SP500"].values)

  # Compute the alpha
  alpha = annualiazed_scalar * (serie.mean() - beta*serie.mean())

  # Print the statistics
  print(f"Sortino: {np.round(sortino,3)}")
  print(f"Beta: {np.round(beta,3)}")
  print(f"Alpha: {np.round(alpha*100,3)} %")
  print(f"MaxDrawdown: {np.round(max_drawdown,3)} %")

In [ ]:
BackTest(ret_pf)